In [2]:
import cv2
import numpy as np 
import os

In [5]:
video_path = 'videos/Banned Pokemon Seizure Scene.mp4'
images_path = 'video1'
if not os.path.isdir(images_path):
    os.mkdir(images_path)

In [8]:
# capture video
video = cv2.VideoCapture(video_path)

# sec -> starting second
def getFrame(sec: int = 0):
    # 2nd parameter converts seconds to miliseconds
    video.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,image = video.read()
    if hasFrames:
        image = cv2.resize(image, dsize=(600, 400))
        cv2.imwrite(images_path + "/img"+str(count)+".jpg", image)     # save frame as JPG file
    return hasFrames

In [9]:
sec = 0
fps = 0.5 #//it will capture image in each 0.5 second
#fps = video.get(cv2.CV_CAP_PROP_FPS)
count=1
success = getFrame(sec)
while getFrame(sec):
    count = count + 1
    sec = sec + fps
    sec = round(sec, 2)
    

In [ ]:
def videos_to_images():
    '''

    '''
    # set starting 

In [2]:
videos = 'videos/'

In [ ]:
# for video in os.listdir(videos):
#     frame = os.path.join(videos, video)
#     frame = frame.resize(frame, dsize=(600, 400))
#     cv2.imshow('Display Image', frame)
#     # 0xff -> escape, 27 escape code as well
#     k = cv2.waitKey(30) & 0xff
#     # if escape key pressed
#     if k == 27:
#         break
# cv2.destroyAllWindows()